# RSTT Tutorial 4 - Implementations Verifications

## StageEvent 

In [1]:
from rstt import Player, BTRanking, BetterWin
from rstt import RoundRobin, SingleEliminationBracket, DoubleEliminationBracket

#from LoL.project.stagedevent import StagedEvent
from project import StagedEvent

# simulation parameters
teams = Player.create(nb=8)
gt = BTRanking(name='GroundTruth', players=teams)
solver = BetterWin()

# competition parameters
tournaments = [RoundRobin, SingleEliminationBracket, DoubleEliminationBracket]
names = ['PlayIns', 'MainStage', 'PlayOffs']

# try your implementation 
test = StagedEvent(name='test', seeding=gt, solver=solver, tournaments=tournaments, stage_names=names)
test.registration(teams)
test.run()

In [2]:
# 1. check stages proper run
for stage in test.stages:
    assert stage.started()
    assert not stage.live()
    assert stage.over()

# 2. check return dataset size
target_games = []
for i, tournament in enumerate(tournaments):
    stage = tournament(name=f'test_stage_games_{i}', seeding=gt, solver=solver)
    stage.registration(teams)
    stage.run()
    target_games += stage.games()
#assert len(test.games()) == len(target_games)

# 3. check final standing
final = tournaments[-1](name='test_final_stage_standing', seeding=gt, solver=solver)
final.registration(teams)
final.run()
assert test.standing() == final.standing()

### LeagueSystem

In [3]:
from project import LeagueSystem
from project.scene import Region

# A LeagueSystem
r_size = 8
teams_by_leagues = {region: Player.create(nb=r_size) for region in Region}
ecosystem = LeagueSystem(teams_by_leagues)

# 1. correct numbers of leagues
assert len(set(ecosystem.leagues())) == len(Region)

# 2. correct number of teams in each region
for region in Region:
    assert len(set(ecosystem.teams(region))) == r_size

# 3. correct total amount of teams
assert len(set(ecosystem.teams()) )== r_size*len(Region)


# 4. correct region of each team
mapping = {league: 0 for league in ecosystem.leagues()}
for region, teams in teams_by_leagues.items():
    for team in teams:
        assert ecosystem.region_of(team) == region
        mapping[ecosystem.league_of(ecosystem.region_of(team))] += 1

# 5. correct number of teams associated to a league
for league in ecosystem.leagues():
    assert mapping[league] == 8


## Game History

In [4]:
from project.gpr import GameHistory
from project.gpr.utils import Modes

# instanciation
LR, TR = 2, 3
history = GameHistory(mode_range={Modes.League: LR, Modes.Team: TR})

# bunch of dummy events
cups = []
for i in range(max(LR, TR)*2):
    cup = RoundRobin(f'history {i}', gt, BetterWin())
    cup.registration(gt.players())
    cup.run()
    cups.append(cup)

    # check ordering of events
    history.add(cup)
    assert cup is history.window(Modes.League)[-1]
    assert cup is history.window(Modes.Team)[-1]

# check number of event returned
assert len(history.window(Modes.League)) == LR
assert len(history.window(Modes.Team)) == TR

# check accessibility of event
for cup in cups:
    assert cup is history.get_event(cup.name())

### Power Score Formula

In [5]:
from rstt.ranking import KeyModel
from project.gpr import RegionalRatings

x = 0.8
y = 0.2
TD, LD = 1500, 1000
datamodel = RegionalRatings(ecosystem,
                            ratings ={
                                Modes.Team: KeyModel(default=TD),
                                Modes.League: KeyModel(default=LD)
                            },
                            x = x, y = y)

# test the GPR scoring formula
for team in ecosystem.teams():
    datamodel.ordinal(team) == x * TD + y * LD

### MetaData

In [6]:
from rstt import GaussianPlayer

from project.model import MetaData
from project.scene import Role

# Instanciation
ROLES = {r: w for r, w in zip(Role, [0.1, 0.25, 0.2, 0.3, 0.15])}
BLevel, RLevel = 1500, 1500
Bs, Rs = 75, 75
BLUE = GaussianPlayer('BlueSide', mu=BLevel, sigma=Bs)
RED = GaussianPlayer('RedSide', mu=RLevel, sigma=Rs)
meta = MetaData(ROLES, BLUE, RED)

# Main feature usage
meta.update()

# check meta changes - side importance
assert BLevel != meta.blue().level()
assert RLevel != meta.red().level()

# check meta changes - role importance
for role in Role:
    assert meta.weights()[role] != ROLES[role]

In [7]:
meta.weights()

{<Role.Toplaner: 'Toplaner'>: 1.680467964711419,
 <Role.Jungle: 'Jungler'>: 0.6055112255831002,
 <Role.Midlaner: 'Midlaner'>: 0.3549520635026653,
 <Role.Botlaner: 'Botlaner'>: 0.5529852194554605,
 <Role.Support: 'Support'>: 1.8060835267473554}

### LoLTeam && LoLSolver

In [9]:
from project.model import LoLTeam
from project.scene import Role

names_A = [f"{str(role)}_A" for role in Role]
names_B = [f"{str(role)}_B" for role in Role]
levels_A = [1000, 1200, 1400, 1600, 1800]
levels_B = [2000, 1700, 1300, 1100, 900]
TeamA = LoLTeam(name="TeamA", )